# Base Models

In [1]:
from importlib import reload

In [2]:
from glob import glob
import os
preproc_train_data_dir = './data/processed/mpqa'
filepaths = glob(os.path.join(preproc_train_data_dir, '*.json'))
len(filepaths)

54

## Bad Sentiment Classifier and Faction Extractor

In [3]:
from src.base_models import sentiment_lexicon
sent_lex_path = 'data/subjectivity_clues_hltemnlp05/subjclueslen1-HLTEMNLP05.tff'
sl, _, _ = sentiment_lexicon.SentimentLexicon.from_mpqa_file(sent_lex_path)

In [59]:
from enum import Enum
import json

from src import util
from src.base_models import bad_sentiment, faction
from src.preproc import entity_extractor

outfiledir = './data/base_model/bad_sentiment_and_faction/mpqa'
print("Classifying features for {} documents".format(len(filepaths)))
for i, infilepath in enumerate(filepaths):
    infilename = os.path.basename(infilepath)
    print("Processing file #{}\t{}".format(i, infilename))
    with open(infilepath, 'r') as infile:
        sentences = json.load(infile)['sentences']
        util.write_deps_to_tokens(sentences)
        ee = entity_extractor.EntityExtractor.from_sentences_with_entity_ids(sentences)
    sentiments_dict = bad_sentiment.classify(sl, sentences, ee)
    factions = list(faction.get_factions(sentences, ee))
        
    sentiments = [
        {
            'holder': holder,
            'target': target,
            'sentiment': sentiment.value
        } for (holder, target), sentiment in sentiments_dict.items()
        if sentiment is not None
    ]
    
    outfilepath = os.path.join(outfiledir, infilename)

    with open(outfilepath, 'w+') as outfile:
        json.dump({
            'docId': infilename,
            'sentiments': sentiments,
            'factions': factions
        }, outfile)

Classifying features for 54 documents
Processing file #0	04.22.14-2532.json
Processing file #1	04.51.05-27505.json
Processing file #2	05.20.33-11163.json
Processing file #3	06.10.04-18139.json
Processing file #4	06.12.31-26764.json
Processing file #5	06.28.56-23638.json
Processing file #6	07.05.30-9348.json
Processing file #7	08.21.04-13527.json
Processing file #8	08.36.15-7509.json
Processing file #9	08.39.09-12713.json
Processing file #10	08.40.56-18707.json
Processing file #11	08.47.00-17401.json
Processing file #12	08.54.29-27700.json
Processing file #13	09.53.15-23595.json
Processing file #14	10.03.26-15373.json
Processing file #15	10.12.58-29108.json
Processing file #16	10.24.29-21670.json
Processing file #17	11.04.20-23621.json
Processing file #18	11.05.55-12013.json
Processing file #19	11.21.37-22256.json
Processing file #20	11.35.22-9439.json
Processing file #21	12.15.47-5091.json
Processing file #22	12.21.28-26118.json
Processing file #23	13.08.06-1812.json
Processing file #2

In [58]:
reload(bad_sentiment);

In [16]:
sorted?